# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [ ]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(cube)
library(data.table)
library(cloudml)
library(readr)
library(logger)
log_threshold(INFO)
setwd("~/")

In [ ]:
# create a Cube object and login
# after run, click on "verification_uri_complete" to finish verification
cube_obj = Cube$new()
cube_obj$login()    

### Get Metadata Definition

In [ ]:
# retrieve metadata element
response = cube_obj$get_element()

In [ ]:
str(response)

In [ ]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [ ]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

### Get Metadata Definition with a Parameter

In [ ]:
response = cube_obj$get_element(element_id = 85)

In [ ]:
str(content(response))

### Get Metadata with accession ids 

In [ ]:
response = cube_obj$get_metadata(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

#### Get Bucket data from metadata collection

In [7]:
storage_info = cube_obj$cubeapi$get_metadata_collection_storage_info()

INFO [2020-12-18 21:47:26] GET: https://metadata-service-sqa.azurewebsites.net/metadata_repository/collection/
INFO [2020-12-18 21:47:28] status_code: 200
INFO [2020-12-18 21:47:28] number of accession_ids: 31
INFO [2020-12-18 21:47:28] GET: https://metadata-service-sqa.azurewebsites.net/metadata_repository/element_instance
INFO [2020-12-18 21:47:31] status_code: 200
INFO [2020-12-18 21:47:31] number of row returned: 6


In [8]:
storage_info

,accession_id,uri,bucket_name,file_name
,<chr>,<chr>,<chr>,<chr>
1,JAXDS00004,gs://jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_NMR_to_201016.txt?authuser=1,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_NMR_to_201016.txt
2,JAXDS00003,https://storage.cloud.google.com/jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_GTT_to_201016.txt?authuser=1,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_GTT_to_201016.txt
3,JAXDS00002,https://storage.cloud.google.com/jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_DEXA_to_201016.txt?authuser=1,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_DEXA_to_201016.txt
4,JAXDS00001,https://storage.cloud.google.com/jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_BODY_WEIGHT_to_201016.txt,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_BODY_WEIGHT_to_201016.txt
5,JAXST00028,https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001188.v1.p1,gs://projects/gap/cgi-bin,study.cgi
6,JAXST00025,https://www.ncbi.nlm.nih.gov/Traces/study/?query_key=11&WebEnv=MCID_5f7c268f0da2dd2639581021&o=acc_s%3Aa,gs://Traces/study,


In [ ]:
## get bucket data
bucket_data = get_bucket_data(bucket_name = storage_info[2, 3], file_name = storage_info[2, 4] )
bucket_data

### Get Metadata Collection

In [ ]:
response = cube_obj$get_metadata_collection()

In [ ]:
json = response_json_to_data(response)

In [ ]:
str(json$results)

In [ ]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

In [ ]:
response = cube_obj$get_metadata(accession_ids = c(accession_ids))

In [ ]:
results = content(response)$results
results[[4]][[4]]

In [ ]:
data_url = results[[4]][[4]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
df

### Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_obj$cubeapi$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_obj$cubeapi$get_end_point("/metadata_repository/collection_item/2/")

In [ ]:
content(response)

### Post endpoint

In [ ]:
response = cube_obj$cubeapi$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### More Information

In [ ]:
?Cube

In [ ]:
?CubeAPI